<a href="https://colab.research.google.com/github/suryareddy1376/DeepLearning/blob/main/CNN_from_Scratch_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
keras.utils.set_random_seed(42)

In [ ]:
df = pd.read_csv("http://storage.googleapis.com/download.tensorflow.org/data/heart.csv")

In [ ]:
df.shape

(303, 14)

In [ ]:
df.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0
5,56,1,2,120,236,0,0,178,0,0.8,1,0,normal,0
6,62,0,4,140,268,0,2,160,0,3.6,3,2,normal,1
7,57,0,4,120,354,0,0,163,1,0.6,1,0,normal,0
8,63,1,4,130,254,0,2,147,0,1.4,2,1,reversible,1
9,53,1,4,140,203,1,2,155,1,3.1,3,0,reversible,0


In [ ]:
df.target.value_counts(normalize=True,dropna=False)

,proportion
target,
0,0.726073
1,0.273927


In [ ]:
categorical_values = ['sex','cp','fbs','restecg','exang','thal','ca']
numeric_values = ['age','trestbps','chol','thalach','oldpeak','slope']

In [ ]:
df = pd.get_dummies(df,columns=categorical_values,dtype=int)

In [ ]:
df.head()

,age,trestbps,chol,thalach,oldpeak,slope,target,sex_0,sex_1,cp_0,...,exang_1,thal_1,thal_2,thal_fixed,thal_normal,thal_reversible,ca_0,ca_1,ca_2,ca_3
0,63,145,233,150,2.3,3,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
1,67,160,286,108,1.5,2,1,0,1,0,...,1,0,0,0,1,0,0,0,0,1
2,67,120,229,129,2.6,2,0,0,1,0,...,1,0,0,0,0,1,0,0,1,0
3,37,130,250,187,3.5,3,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
4,41,130,204,172,1.4,1,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0


In [ ]:
test_df = df.sample(frac=.2,random_state=42)
train_df = df.drop(test_df.index)

##Normalization

In [ ]:
means = train_df[numeric_values].mean()
sd = train_df[numeric_values].std()

In [ ]:
train_df =